In [1]:
from collections import namedtuple
from dataclasses import dataclass
from enum import Enum
import heapq as heap
from copy import copy
from typing import Optional, List
import numpy as np
from IPython.display import HTML

from large_gcs.graph.incremental_relaxed_contact_graph import IncrementalRelaxedContactGraph
from large_gcs.graph.incremental_contact_graph import IncrementalContactGraph
from large_gcs.graph.contact_graph import ContactGraph
from large_gcs.graph.graph import ShortestPathSolution, Graph
from large_gcs.graph_generators.contact_graph_generator import ContactGraphGeneratorParams
from large_gcs.graph.factored_collision_free_graph import FactoredCollisionFreeGraph
from large_gcs.cost_estimators.factored_collision_free_ce import FactoredCollisionFreeCE

import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("drake").setLevel(logging.ERROR)

In [2]:
graph_name = "cg_radialsort_a1"
graph_file = ContactGraphGeneratorParams.inc_graph_file_path_from_name(graph_name)
cg_inc = IncrementalContactGraph.load_from_file(graph_file, should_add_gcs=True)
cg_inc_relaxed = IncrementalRelaxedContactGraph.load_from_file(graph_file, should_add_gcs=True, should_incl_simul_mode_switches=False)

INFO:large_gcs.graph.contact_graph:Creating vertex costs...


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-01


100%|██████████| 2/2 [00:00<00:00, 12087.33it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 13025.79it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 12 body pairs...
100%|██████████| 12/12 [00:00<00:00, 69.00it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (345)
100%|██████████| 345/345 [00:00<00:00, 6150.69it/s]
INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 27685.17it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 21564.54it/s]
INFO:large_gcs.graph.incremental_relaxed_contact_graph:Generating contact pair modes for 12 body pairs...
100%|██████████| 3/3 [00:00<00:00, 114.94it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (264)
100%|██████████| 264/264 [00:00<00:00, 6994.70it/s]


In [3]:
full_v_name = cg_inc.vertex_names[2]
rc_v_name = cg_inc_relaxed.vertex_names[2]

In [4]:
print(full_v_name)
print(rc_v_name)

('NC|obs0_f1-obj0_v2', 'NC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'NC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'NC|obj0_f1-obj1_f3', 'NC|obj0_f0-rob0_f2', 'NC|obj1_f0-rob0_f2')
('NC|obs0_f1-obj0_v2', 'NC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'NC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'RNC|obj0-obj1', 'RNC|obj0-rob0', 'RNC|obj1-rob0')


In [9]:
print(rc_v_name)
print(IncrementalRelaxedContactGraph.full_to_relaxed_contact_vertex_name(full_v_name))

('NC|obs0_f1-obj0_v2', 'NC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'NC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'RNC|obj0-obj1', 'RNC|obj0-rob0', 'RNC|obj1-rob0')
('NC|obs0_f1-obj0_v2', 'NC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'NC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'RNC|obj0-obj1', 'RNC|obj0-rob0', 'RNC|obj1-rob0')


In [6]:
import re

def convert_string(s):
    # This regex will match the undesired parts, like _f1, _v2, or their combinations with other characters
    pattern = r'(_f\d+|_v\d+|\_f\d+_\w+|\_v\d+_\w+)'
    # This will replace those parts with an empty string
    return re.sub(pattern, '', s)

# Test the function
test_strings = ['NC|obs2_f1-rob0_v2', 'NC|obs1_v2-rob0_f32']
converted_strings = [convert_string(s) for s in test_strings]

for original, converted in zip(test_strings, converted_strings):
    print(f'{original} -> {converted}')


NC|obs2_f1-rob0_v2 -> NC|obs2-rob0
NC|obs1_v2-rob0_f32 -> NC|obs1-rob0


In [7]:
test = "('IC|obs0_f1-obj0_v2', 'IC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'IC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'IC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'IC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'IC|obj0_v31-obj1_f33', 'NC|obj0_f0-rob0_f2', 'IC|obj1_f230-rob0_v2')"
print(test)
rc_res = IncrementalRelaxedContactGraph.full_to_relaxed_contact_vertex_name(test)
print(rc_res)

('IC|obs0_f1-obj0_v2', 'IC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'IC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'IC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'IC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'IC|obj0_v31-obj1_f33', 'NC|obj0_f0-rob0_f2', 'IC|obj1_f230-rob0_v2')
('IC|obs0_f1-obj0_v2', 'IC|obs0_f2-obj1_v3', 'NC|obs0_f1-rob0_v2', 'IC|obs1_f2-obj0_v3', 'NC|obs1_f2-obj1_v3', 'IC|obs1_f2-rob0_v3', 'NC|obs2_f1-obj0_v2', 'IC|obs2_f1-obj1_v2', 'NC|obs2_f1-rob0_v2', 'RIC|obj0-obj1', 'RNC|obj0-rob0', 'RIC|obj1-rob0')


In [8]:
print(FactoredCollisionFreeCE.convert_to_cfree_vertex_names(test))
print(FactoredCollisionFreeCE.convert_to_cfree_vertex_names(rc_res))

["('NC|obs0_f1-obj0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs2_f1-obj0_v2')", "('NC|obs0_f2-obj1_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs2_f1-obj1_v2')", "('NC|obs0_f1-rob0_v2', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-rob0_v2')"]
["('NC|obs0_f1-obj0_v2', 'NC|obs1_f2-obj0_v3', 'NC|obs2_f1-obj0_v2')", "('NC|obs0_f2-obj1_v3', 'NC|obs1_f2-obj1_v3', 'NC|obs2_f1-obj1_v2')", "('NC|obs0_f1-rob0_v2', 'NC|obs1_f2-rob0_v3', 'NC|obs2_f1-rob0_v2')"]
